# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

## Reading the CSV and Performing Basic Data Cleaning

In [3]:
file_path = Path('Resources/LoanStats_2019Q1.csv')

df = pd.read_csv(file_path)

Raw Data Preview

In [4]:
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

Let's reorganize out dataframe so that our target variable is set as the last column and all our features come before it.

In [5]:
df = df[[c for c in df if c not in ['loan_status']] + ['loan_status']]
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,loan_status
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,n,27.24,0.0,...,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N,low_risk
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,n,20.23,0.0,...,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N,low_risk
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,n,24.26,0.0,...,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N,low_risk
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,n,31.44,0.0,...,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N,low_risk
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,n,18.76,0.0,...,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N,low_risk


Now, let's separate our feature variables from our target variable observations into separate dataframes

In [6]:
X =  df.iloc[:, 0:-1]

y = df.iloc[:,-1:]

X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,n,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,n,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,n,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,n,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,n,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


Let's also integer encode our features observations

In [7]:
X = pd.get_dummies(X)

X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


Checking the balance of our target values - Not really a balanced dataset, right?

In [8]:
y.value_counts()

loan_status
low_risk       68470
high_risk        347
dtype: int64

:et's split the X and y datasets in 4 subsets: X_train, X_test, y_train, y_test

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=2, 
                                                    stratify=y)

## Data Pre-Processing

Instantiating a scaler object in order to standardize our features dataset

In [10]:
scaler = StandardScaler()


In [25]:
X_scaler=scaler.fit(X_train)           

In [24]:
X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)


## Ensemble Learners

In this section, we will compare two ensemble algorithms to determine which algorithm results in the best performance. We  will train a Balanced Random Forest Classifier and an Easy Ensemble classifier. For each algorithm, be sure to complete the folliowing steps:


### Balanced Random Forest Classifier

Resampling the training data with the BalancedRandomForestClassifier and making our first predictions

In [13]:

rf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=1)

rf_model = rf_model.fit(X_train_scaled, y_train)

predictions_rf = rf_model.predict(X_test_scaled)

### Performance metrics

Balanced Accuracy Score

In [14]:
bal_acc_score = balanced_accuracy_score(y_test, predictions_rf)
print(bal_acc_score)

0.8314367614650439


Confusion Matrix

In [20]:
cm = confusion_matrix(y_test, predictions_rf)

cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

Accuracy score

In [19]:
acc_score = accuracy_score(y_test, predictions_rf)
print(acc_score)

0.9035164196454519


Classification Report-Imbalanced

In [21]:

print(classification_report_imbalanced(y_test, predictions_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.76      0.90      0.07      0.83      0.68        87
   low_risk       1.00      0.90      0.76      0.95      0.83      0.70     17118

avg / total       0.99      0.90      0.76      0.94      0.83      0.70     17205



List the features sorted in descending order by feature importance

In [23]:
importances = rf_model.feature_importances_

importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

importances_sorted[:10]

[(0.06331044751036251, 'total_rec_prncp'),
 (0.0608718088096818, 'total_rec_int'),
 (0.05767339870962936, 'total_pymnt_inv'),
 (0.05117499995490403, 'last_pymnt_amnt'),
 (0.0507633263634394, 'total_pymnt'),
 (0.028226854829184248, 'int_rate'),
 (0.025612206336332644, 'issue_d_Jan-2019'),
 (0.01882024673095996, 'installment'),
 (0.017824423474424703, 'max_bal_bc'),
 (0.017098031044373754, 'annual_inc')]

### Easy Ensemble Classifier

Instantiating our new easy ensemble model and making new predictions

In [27]:
model_easy = EasyEnsembleClassifier()

model_easy.fit(X_train_scaled, y_train)

y_pred_easy = model_easy.predict(X_test_scaled)


### Performance metrics

Balanced accuracy score

In [ ]:

bal_acc_score_easy = balanced_accuracy_score(y_test, y_pred_easy)
print(bal_acc_score_easy)


Confusion matrix

In [28]:
print(confusion_matrix(y_test, y_pred_easy))

[[   79     8]
 [ 1059 16059]]


Imbalanced classification report

In [ ]:

print(classification_report_imbalanced(y_test, y_pred_easy))

List the features sorted in descending order by feature importance

In [30]:
importances_easy = rf_model.feature_importances_

importances_easy_sorted= sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
print(importances_easy_sorted[:5])

[(0.06331044751036251, 'total_rec_prncp'), (0.0608718088096818, 'total_rec_int'), (0.05767339870962936, 'total_pymnt_inv'), (0.05117499995490403, 'last_pymnt_amnt'), (0.0507633263634394, 'total_pymnt')]


### Conclusions

1. Which model had the best balanced accuracy score?

- The model with the highest balanced accuracy score was the Easy Ensemble Classifier with a high score of over 0.92

2. Which model had the best recall score?

 - Again, our Easy Ensembler classifier model scored 0.95 average recall score.

3. Which model had the best geometric mean score?

 - Our EasyEnsembler model had a far superios geometric mean score of 0.92 vs. 0.83 from our Balanced Random Forest classifier.

4. What are the top three features?

- Not surprisingly "total received principal" has the highest predictive power among all features, followed by total received interest and total payment inv.